In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor as cbr
from datetime import datetime as dt
from lightgbm import LGBMRegressor as lbr
from xgboost import XGBRegressor as xgb
from sklearn.ensemble import RandomForestRegressor as rf

In [2]:
data=pd.read_excel("Hackathon Data.xlsx")
data.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,?,498,Fully-Stocked,6
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7


In [3]:
data.shape

(846632, 11)

In [6]:
inflation=pd.read_excel("Inflation_rates.xlsx")

In [46]:
inf=inflation.copy()
inf = inf.drop(['HALF1', 'HALF2'], axis=1)
year_cols = inf.columns.tolist()[1:]

inf = pd.melt(inf, id_vars=['Year'], value_vars=year_cols)

inf_filtered = inf[inf['Year']>=2017]
inf_filtered.reset_index(drop=True)

,Year,variable,value
0,2017,Jan,2.3
1,2018,Jan,1.8
2,2019,Jan,2.2
3,2020,Jan,2.3
4,2021,Jan,1.4
...,...,...,...
67,2018,Dec,2.2
68,2019,Dec,2.3
69,2020,Dec,1.6
70,2021,Dec,5.5


In [48]:
month_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

inf_filtered['month'] = inf_filtered['variable'].apply(lambda x: month_dict[x])

C:\Users\ch14b\AppData\Local\Temp\ipykernel_10180\1781634866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inf_filtered['month'] = inf_filtered['variable'].apply(lambda x: month_dict[x])


,Year,variable,value,month
5,2017,Jan,2.3,1
6,2018,Jan,1.8,1
7,2019,Jan,2.2,1
8,2020,Jan,2.3,1
9,2021,Jan,1.4,1
...,...,...,...,...
127,2018,Dec,2.2,12
128,2019,Dec,2.3,12
129,2020,Dec,1.6,12
130,2021,Dec,5.5,12


In [50]:
inf_filtered = inf_filtered.drop('variable', axis=1)
inf_filtered

,Year,value,month
5,2017,2.3,1
6,2018,1.8,1
7,2019,2.2,1
8,2020,2.3,1
9,2021,1.4,1
...,...,...,...
127,2018,2.2,12
128,2019,2.3,12
129,2020,1.6,12
130,2021,5.5,12


In [20]:
df=data.copy()

In [21]:
df.replace('?', np.nan, inplace=True)

In [86]:
df[df['Encoded_SKU_ID']==398].tail()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,Movingvar5,Movingvar3,PROMO_PRICE_BIN,COMPETITOR_PRICE_BIN,Movingsum7_BIN,Movingsum5_BIN,Movingsum3_BIN,Movingvar7_BIN,Movingvar5_BIN,Movingvar3_BIN
626698,398,2022-07-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,7.99,0.0,0.0,Fully-Stocked,...,3.7,4.0,1,1,0,0,0,0,0,0
626699,398,2022-07-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,7.99,0.0,0.0,Fully-Stocked,...,3.7,4.0,1,1,0,0,0,0,0,0
626700,398,2022-07-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,7.99,0.0,0.0,Fully-Stocked,...,3.7,4.0,1,1,0,0,0,0,0,0
626701,398,2022-07-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,7.99,0.0,0.0,Fully-Stocked,...,3.7,4.0,1,1,0,0,0,0,0,0
626702,398,2022-07-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,7.99,0.0,0.0,Fully-Stocked,...,3.7,4.0,1,1,0,0,0,0,0,0


In [23]:
df.dtypes

Encoded_SKU_ID               int64
SALES_DATE          datetime64[ns]
SUBCLASS_NAME               object
CLASS_NAME                  object
ML_NAME                     object
CATEGORY_NAME               object
RETAIL_PRICE               float64
PROMO_PRICE                float64
COMPETITOR_PRICE           float64
Inventory                   object
DAILY_UNITS                  int64
dtype: object

In [24]:
df['day']=df['SALES_DATE'].dt.day
df['month']=df['SALES_DATE'].dt.month
df['year']=df['SALES_DATE'].dt.year
df['day_of_week']=df['SALES_DATE'].dt.day_name()

In [25]:
df['DAILY_UNITS'].describe()

count    846632.000000
mean         10.532819
std          70.124984
min          -4.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       19136.000000
Name: DAILY_UNITS, dtype: float64

In [26]:
df.nunique()

Encoded_SKU_ID       575
SALES_DATE          1891
SUBCLASS_NAME        116
CLASS_NAME            25
ML_NAME                8
CATEGORY_NAME          3
RETAIL_PRICE         509
PROMO_PRICE          635
COMPETITOR_PRICE    5556
Inventory              4
DAILY_UNITS         1465
day                   31
month                 12
year                   6
day_of_week            7
dtype: int64

In [27]:
def func(x):
    x=x.sort_values("SALES_DATE")
    x["Movingsum7"]=x['DAILY_UNITS'].rolling(window=7).sum().shift(1)
    x["Movingsum5"]=x['DAILY_UNITS'].rolling(window=5).sum().shift(1)
    x["Movingsum3"]=x['DAILY_UNITS'].rolling(window=3).sum().shift(1)
    x["Movingvar7"]=x['DAILY_UNITS'].rolling(window=7).var().shift(1)
    x["Movingvar5"]=x['DAILY_UNITS'].rolling(window=5).var().shift(1)
    x["Movingvar3"]=x['DAILY_UNITS'].rolling(window=3).var().shift(1)
    cols=[i for i in x.columns if ("Moving" in i) and ("BIN" not in i)]
    x.reset_index(inplace=True,drop=True)
    x.loc[int(0.9*len(x)):,cols]=np.nan
    x[cols]=x[cols].fillna(method="ffill")
    return x

In [28]:
df=df.groupby("Encoded_SKU_ID",as_index=False).apply(func).sort_values(["Encoded_SKU_ID","SALES_DATE"]).reset_index(drop=True)

In [31]:
df.head(10)

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,day,month,year,day_of_week,Movingsum7,Movingsum5,Movingsum3,Movingvar7,Movingvar5,Movingvar3
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,28,5,2017,Sunday,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,29,5,2017,Monday,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,30,5,2017,Tuesday,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,31,5,2017,Wednesday,NaN,NaN,0.0,NaN,NaN,0.000000
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,1,6,2017,Thursday,NaN,NaN,0.0,NaN,NaN,0.000000
5,1,2017-06-02,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,2,6,2017,Friday,NaN,0.0,0.0,NaN,0.0,0.000000
6,1,2017-06-03,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,3,6,2017,Saturday,NaN,4.0,4.0,NaN,3.2,5.333333
7,1,2017-06-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,4,6,2017,Sunday,11.0,11.0,11.0,7.952381,10.2,12.333333
8,1,2017-06-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,5,6,2017,Monday,15.0,15.0,15.0,8.142857,9.0,3.000000
9,1,2017-06-06,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,...,6,6,2017,Tuesday,17.0,17.0,13.0,7.285714,6.8,6.333333


In [51]:
df['PROMO_PRICE_BIN'] = df["PROMO_PRICE"].isnull().astype(int)
df['COMPETITOR_PRICE_BIN'] = df["COMPETITOR_PRICE"].isnull().astype(int)
df['Movingsum7_BIN'] = df["Movingsum7"].isnull().astype(int)
df['Movingsum5_BIN'] = df["Movingsum5"].isnull().astype(int)
df['Movingsum3_BIN'] = df["Movingsum3"].isnull().astype(int)
df['Movingvar7_BIN'] = df["Movingvar7"].isnull().astype(int)
df['Movingvar5_BIN'] = df["Movingvar5"].isnull().astype(int)
df['Movingvar3_BIN'] = df["Movingvar3"].isnull().astype(int)
df=df.fillna(0)
df.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,Movingvar5,Movingvar3,PROMO_PRICE_BIN,COMPETITOR_PRICE_BIN,Movingsum7_BIN,Movingsum5_BIN,Movingsum3_BIN,Movingvar7_BIN,Movingvar5_BIN,Movingvar3_BIN
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,0,1,1,0
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,0,1,1,0


In [52]:
df=df.sort_values("SALES_DATE")

In [53]:
covid=pd.read_csv("owid-covid-data.csv")
covid_us=covid[covid["location"]=="United States"]

In [54]:
covid_us["date"]=pd.to_datetime(covid_us["date"])

C:\Users\ch14b\AppData\Local\Temp\ipykernel_10180\3481439096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_us["date"]=pd.to_datetime(covid_us["date"])


In [55]:
covid_us_filtered = covid_us[['date', 'total_cases']]
covid_us_filtered.head()

,date,total_cases
233448,2020-01-22,1.0
233449,2020-01-23,1.0
233450,2020-01-24,2.0
233451,2020-01-25,2.0
233452,2020-01-26,5.0


In [58]:
merged=pd.merge(df,covid_us_filtered,how="left",left_on="SALES_DATE",right_on="date")
merged.drop('date', axis=1, inplace=True)
merged = merged.fillna(0)

merged1 = pd.merge(merged, inf_filtered, how='left', left_on=['year', 'month'],right_on=['Year', 'month'])
merged1.drop(['Year'], axis=1, inplace=True)

merged1.tail()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,PROMO_PRICE_BIN,COMPETITOR_PRICE_BIN,Movingsum7_BIN,Movingsum5_BIN,Movingsum3_BIN,Movingvar7_BIN,Movingvar5_BIN,Movingvar3_BIN,total_cases,value
846627,84,2022-07-31,CC MICROPHONES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,99.99,0.00,88.00,Fully-Stocked,...,1,0,0,0,0,0,0,0,91362615.0,5.9
846628,401,2022-07-31,SO CAR ACCESSORIES,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,0.00,0.00,Fully-Stocked,...,1,1,0,0,0,0,0,0,91362615.0,5.9
846629,85,2022-07-31,FRENCH DOOR REFRIG,REFRIGERATION,ML - Kitchen,VP - Major Appliances,2899.99,2599.99,2598.00,Fully-Stocked,...,0,0,0,0,0,0,0,0,91362615.0,5.9
846630,406,2022-07-31,DECK HARNESS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,43.99,0.00,0.00,Fully-Stocked,...,1,1,0,0,0,0,0,0,91362615.0,5.9
846631,575,2022-07-31,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,0.00,124.95,Fully-Stocked,...,1,0,0,0,0,0,0,0,91362615.0,5.9


In [59]:
cat_feats=merged1.columns[np.where(merged1.dtypes == "object")[0]].values.tolist()
# cat_feats.insert(0,"Encoded_SKU_ID")
merged1[merged1.columns[np.where(merged1.dtypes == object)[0]].values.tolist()]=merged1[merged1.columns[np.where(merged1.dtypes == object)[0]].values.tolist()].astype('str')

bincols=[i for i in cat_feats if "BIN" in cat_feats]

cat_feats+=['PROMO_PRICE_BIN', 'COMPETITOR_PRICE_BIN']
cat_feats

['SUBCLASS_NAME',
 'CLASS_NAME',
 'ML_NAME',
 'CATEGORY_NAME',
 'Inventory',
 'day_of_week',
 'PROMO_PRICE_BIN',
 'COMPETITOR_PRICE_BIN']

In [61]:
filcols= [i for i in merged1.columns if ("3_BIN" not in i)]
filcols = [i for i in filcols if ("5_BIN" not in i)]
filcols = [i for i in filcols if ("7_BIN" not in i)]
filcols

['Encoded_SKU_ID',
 'SALES_DATE',
 'SUBCLASS_NAME',
 'CLASS_NAME',
 'ML_NAME',
 'CATEGORY_NAME',
 'RETAIL_PRICE',
 'PROMO_PRICE',
 'COMPETITOR_PRICE',
 'Inventory',
 'DAILY_UNITS',
 'day',
 'month',
 'year',
 'day_of_week',
 'Movingsum7',
 'Movingsum5',
 'Movingsum3',
 'Movingvar7',
 'Movingvar5',
 'Movingvar3',
 'PROMO_PRICE_BIN',
 'COMPETITOR_PRICE_BIN',
 'total_cases',
 'value']

In [62]:
merged2=merged1[filcols]

In [63]:
from sklearn.model_selection import train_test_split

In [149]:
xtr1,xts1,ytr1,yts1 = train_test_split(df.drop(["DAILY_UNITS"], axis=1), df["DAILY_UNITS"], shuffle=False, test_size=0.1)

reg=cbr(cat_features=cat_feats,depth=2,num_trees=2000)
reg.fit(xtr1,ytr1)
reg.score(xts1, yts1),reg.score(xtr1,ytr1)

Learning rate set to 0.066495
0:	learn: 70.3089055	total: 113ms	remaining: 3m 45s
1:	learn: 67.9634430	total: 192ms	remaining: 3m 11s
2:	learn: 65.8818847	total: 288ms	remaining: 3m 11s
3:	learn: 63.9012606	total: 391ms	remaining: 3m 14s
4:	learn: 62.1045202	total: 469ms	remaining: 3m 7s
5:	learn: 60.5085722	total: 544ms	remaining: 3m
6:	learn: 58.9577680	total: 619ms	remaining: 2m 56s
7:	learn: 57.5764100	total: 697ms	remaining: 2m 53s
8:	learn: 56.3153881	total: 779ms	remaining: 2m 52s
9:	learn: 55.1446234	total: 857ms	remaining: 2m 50s
10:	learn: 54.1229370	total: 945ms	remaining: 2m 50s
11:	learn: 53.1253543	total: 1.04s	remaining: 2m 51s
12:	learn: 52.2386338	total: 1.11s	remaining: 2m 50s
13:	learn: 51.5713717	total: 1.21s	remaining: 2m 51s
14:	learn: 50.7945139	total: 1.29s	remaining: 2m 50s
15:	learn: 50.1311238	total: 1.4s	remaining: 2m 53s
16:	learn: 49.4847028	total: 1.5s	remaining: 2m 54s
17:	learn: 48.9024117	total: 1.59s	remaining: 2m 55s
18:	learn: 48.4789384	total: 1.71

(-0.08150400598280694, 0.8134878645130549)

In [64]:
merged3 = pd.get_dummies(merged2, columns = merged2.columns[np.where(merged.dtypes == "object")[0]].values.tolist())

In [65]:
merged3.dtypes

Encoded_SKU_ID                    int64
SALES_DATE               datetime64[ns]
RETAIL_PRICE                    float64
PROMO_PRICE                     float64
COMPETITOR_PRICE                float64
                              ...      
day_of_week_Saturday              uint8
day_of_week_Sunday                uint8
day_of_week_Thursday              uint8
day_of_week_Tuesday               uint8
day_of_week_Wednesday             uint8
Length: 182, dtype: object

In [80]:
xtr,xts,ytr,yts = train_test_split(merged3.drop(["DAILY_UNITS", 'SALES_DATE'], axis=1), merged3["DAILY_UNITS"], shuffle=False, test_size=0.1)

In [161]:
#Before Inflation results
# reg=lbr(max_depth=2,num_leaves=2)
# reg.fit(xtr,ytr)
# reg.score(xts, yts),reg.score(xtr,ytr)

(0.31091677464672196, 0.6555274389464999)

In [69]:
#After Inflation Results
reg_i=lbr(max_depth=2,num_leaves=2)
reg_i.fit(xtr,ytr)
reg_i.score(xts, yts),reg_i.score(xtr,ytr)

(0.31091677464672196, 0.6555274389464999)

In [182]:
reg1=lbr(num_leaves=2, reg_lambda=0.999, reg_alpha=0.999)
reg1.fit(xtr,ytr)
reg1.score(xts, yts),reg1.score(xtr,ytr)

(0.32378332003177657, 0.655511580124942)

In [187]:
reg20=xgb(max_depth=2, gamma=5)
reg20.fit(xtr,ytr)
reg20.score(xts, yts),reg20.score(xtr,ytr)

(0.20238659928551694, 0.8986892472008448)

In [188]:
reg21=xgb(max_depth=2, gamma=5, eta=0.05)
reg21.fit(xtr,ytr)
reg21.score(xts, yts),reg21.score(xtr,ytr)

(0.3678334764138318, 0.7815045291886457)

In [81]:
reg22=xgb(max_depth=2, gamma=5, eta=0.01)
reg22.fit(xtr,ytr)
reg22.score(xts, yts),reg22.score(xtr,ytr)

(0.5325342515308715, 0.5623917992002656)

In [ ]:
reg=cbr(cat_features=cat_feats,depth=2,num_trees=2000)
reg.fit(xtr,ytr)
reg4.score(xts, yts),reg4.score(xtr,ytr)

In [137]:
reg5=cbr(cat_features=cat_feats,depth=2,num_trees=500)
reg5.fit(xtr,ytr)
reg5.score(xts, yts),reg5.score(xtr,ytr)

Learning rate set to 0.20524
0:	learn: 65.2757635	total: 103ms	remaining: 51.4s
1:	learn: 59.5559229	total: 194ms	remaining: 48.3s
2:	learn: 55.3708115	total: 310ms	remaining: 51.3s
3:	learn: 52.1252178	total: 393ms	remaining: 48.7s
4:	learn: 50.1327530	total: 491ms	remaining: 48.6s
5:	learn: 48.2593801	total: 571ms	remaining: 47s
6:	learn: 46.7534951	total: 648ms	remaining: 45.6s
7:	learn: 45.8091839	total: 734ms	remaining: 45.1s
8:	learn: 44.9939697	total: 813ms	remaining: 44.4s
9:	learn: 44.3750823	total: 895ms	remaining: 43.9s
10:	learn: 43.9614937	total: 981ms	remaining: 43.6s
11:	learn: 43.6105568	total: 1.08s	remaining: 43.8s
12:	learn: 43.3622359	total: 1.17s	remaining: 43.9s
13:	learn: 43.2486518	total: 1.27s	remaining: 44s
14:	learn: 43.0845143	total: 1.35s	remaining: 43.8s
15:	learn: 42.9268515	total: 1.45s	remaining: 43.9s
16:	learn: 42.8161308	total: 1.53s	remaining: 43.6s
17:	learn: 42.7395377	total: 1.61s	remaining: 43.2s
18:	learn: 42.6546394	total: 1.69s	remaining: 42.

(-0.5211171817913951, 0.8158672344701392)